## Structured Output Parser
`Structured Output Parser`란?
- LLM의 출력을 구조화된 딕셔너리 형태로 파싱하는 도구입니다
- ResponseSchema를 사용하여 원하는 출력 구조를 정의합니다
- Pydantic보다 간단하고 CommaSeparatedList보다 구조화된 중간 수준의 파서입니다
- 키-값 쌍으로 이루어진 구조화된 데이터를 반환합니다

LLM 모델 객체 생성하기

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.1,  # 구조화된 출력을 위해 낮은 temperature 설정
)

구조화된 정보 추출이 필요할 때,  
`StructuredOutputParser`를 사용하면 텍스트를 자동으로 딕셔너리 형태로 변환할 수 있습니다.

### ResponseSchema란?
- LLM의 출력을 구조화하기 위한 스키마 정의 클래스
- 출력 필드의 이름(name)과 설명(description)을 정의
- 각 필드가 어떤 정보를 담아야 하는지 LLM에게 안내하는 역할

1. ResponseSchema 정의 및 Structured Output Parser 생성하기

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# ResponseSchema 정의 - 각 필드의 이름과 설명을 정의
response_schemas = [
    ResponseSchema(name="name", description="군인의 이름"),
    ResponseSchema(name="rank", description="군인의 계급 (예: 이병, 일병, 상병, 병장)"),
    ResponseSchema(name="unit", description="소속 부대명"),
    ResponseSchema(name="specialty", description="특기 또는 병과"),
    ResponseSchema(name="evaluation", description="성과 평가 (우수, 보통, 미흡)")
]

# Structured Output Parser 생성
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 파서가 어떤 형식을 요구하는지 확인
print(parser.get_format_instructions())


- **get_format_instructions()**: ResponseSchema를 기반으로 LLM이 이해할 수 있는 출력 형식 지침을 생성
- LLM이 지정된 구조에 맞춰 JSON 형태로 응답하도록 안내

- `soldier_info` : 분석할 군인 정보 (사용자 입력을 받을 위치)
- `format_instructions` : parser로 형식 지정

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 재정의 (format_instructions 포함)
template = """
다음 군인 정보를 분석하여 구조화된 형태로 정리해주세요.

군인 정보: {soldier_info}

{format_instructions}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["soldier_info"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 프롬프트 미리보기
formatted_prompt = prompt.format(soldier_info="[soldier_info]")
print(f"생성된 프롬프트 템플릿:{formatted_prompt}")


LCEL 체인 만들기
- 체인 구조: PromptTemplate → ChatOllama → StructuredOutputParser

In [ ]:
# LCEL을 사용한 체인 생성: 프롬프트 → LLM → 파서
chain = prompt | llm | parser

# 테스트 데이터
test_info = "이동훈 상병은 제3기계화보병사단 소속 정비병으로 18개월 복무했으며 매우 우수한 평가를 받고 있습니다."

# 체인 실행
result = chain.invoke({"soldier_info": test_info})

print(f"파싱 결과: {result}")
print(f"이름: {result.get('name')}")
